## Data Preparation (1)
- create derived variables
 - Euclidean distance from the origin
 - atomic information from the periodic table
 - bond information
- sample demonstration

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore')

In [2]:
from glob import glob
from tqdm import tqdm

In [3]:
path = 'data/'

sample = pd.read_csv(path + 'sample_submission.csv')
train = pd.read_csv(path + 'train_set.ReorgE.csv')
test = pd.read_csv(path + 'test_set.csv')

## Sample Demonstration on Creating Derived Variables
- Euclidean distance from the origin
- atomic information
 - numerical data from the periodic table
- bond information
 - bond length
 - bond energy

### (1) Distance from the Origin

In [4]:
path = 'data/mol_files/train_set/'

df_a = pd.read_csv(path + 'train_0_g_a.csv')
df_a.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2.8972,-2.3256,0.4309,C,0,0,0,0,0,0,0,0,0,0,0,0
1,3.3467,-1.0151,1.0796,C,0,0,0,0,0,0,0,0,0,0,0,0
2,3.3365,0.2041,0.1293,C,0,0,1,0,0,0,0,0,0,0,0,0
3,1.9424,0.6155,-0.3895,C,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0870,1.3235,0.6658,C,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df_a['dist'] = np.sqrt(df_a['0']**2 + df_a['1']**2 + df_a['2']**2)
df_a.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,dist
0,2.8972,-2.3256,0.4309,C,0,0,0,0,0,0,0,0,0,0,0,0,3.740034
1,3.3467,-1.0151,1.0796,C,0,0,0,0,0,0,0,0,0,0,0,0,3.660105
2,3.3365,0.2041,0.1293,C,0,0,1,0,0,0,0,0,0,0,0,0,3.345237
3,1.9424,0.6155,-0.3895,C,0,0,0,0,0,0,0,0,0,0,0,0,2.074480
4,1.0870,1.3235,0.6658,C,0,0,0,0,0,0,0,0,0,0,0,0,1.837528


### (2) Atomic Information

In [6]:
# train_set

path = 'data/mol_files/train_set/'

symbols = []

for i in tqdm(train['index']):
    
    at_g = pd.read_csv(path + i + '_g_a.csv')
        
    for j in at_g['3']:
        symbols.append(j)

100%|███████████████████████████████████████████████████████████████████████████| 18157/18157 [01:07<00:00, 270.76it/s]


In [7]:
# test_set

path = 'data/mol_files/test_set/'

for i in tqdm(test['index']):
    
    at_g = pd.read_csv(path + i + '_g_a.csv')
        
    for j in at_g['3']:
        symbols.append(j)

100%|███████████████████████████████████████████████████████████████████████████████| 457/457 [00:01<00:00, 255.02it/s]


In [8]:
symbols = list(set(symbols))
symbols

['S', 'P', 'H', 'I', 'Br', 'F', 'N', 'Cl', 'C', 'O', 'B', 'Si']

In [9]:
table = pd.read_csv('https://pubchem.ncbi.nlm.nih.gov/rest/pug/periodictable/CSV/?response_type=display')
table.head().T

,0,1,2,3,4
AtomicNumber,1,2,3,4,5
Symbol,H,He,Li,Be,B
Name,Hydrogen,Helium,Lithium,Beryllium,Boron
AtomicMass,1.008,4.0026,7.0,9.012183,10.81
CPKHexColor,FFFFFF,D9FFFF,CC80FF,C2FF00,FFB5B5
ElectronConfiguration,1s1,1s2,[He]2s1,[He]2s2,[He]2s2 2p1
Electronegativity,2.2,NaN,0.98,1.57,2.04
AtomicRadius,120.0,140.0,182.0,153.0,192.0
IonizationEnergy,13.598,24.587,5.392,9.323,8.298
ElectronAffinity,0.754,NaN,0.618,NaN,0.277


In [10]:
table.index = table['Symbol']
tmp = table.loc[symbols].reset_index(drop=True)
tmp

##### numerical variables #####
# AtomicNumber, AtomicMass, Electronegativity, IonizationEnergy,
# ElectronAffinity, MeltingPoint, BoilingPoint, Density
# ** use 'bond length' instead of 'atomic radius'

##### categorical variables #####
# StandardState, GroupBlock

,AtomicNumber,Symbol,Name,AtomicMass,CPKHexColor,ElectronConfiguration,Electronegativity,AtomicRadius,IonizationEnergy,ElectronAffinity,OxidationStates,StandardState,MeltingPoint,BoilingPoint,Density,GroupBlock,YearDiscovered
0,16,S,Sulfur,32.070000,FFFF30,[Ne]3s2 3p4,2.58,180.0,10.360,2.077,"+6, +4, -2",Solid,388.36,717.75,2.067000,Nonmetal,Ancient
1,15,P,Phosphorus,30.973762,FF8000,[Ne]3s2 3p3,2.19,180.0,10.487,0.746,"+5, +3, -3",Solid,317.30,553.65,1.820000,Nonmetal,1669
2,1,H,Hydrogen,1.008000,FFFFFF,1s1,2.20,120.0,13.598,0.754,"+1, -1",Gas,13.81,20.28,0.000090,Nonmetal,1766
3,53,I,Iodine,126.904500,940094,[Kr]5s2 4d10 5p5,2.66,198.0,10.451,3.059,"+7, +5, +1, -1",Solid,386.85,457.55,4.930000,Halogen,1811
4,35,Br,Bromine,79.900000,A62929,[Ar]4s2 3d10 4p5,2.96,183.0,11.814,3.365,"+5, +1, -1",Liquid,265.95,331.95,3.110000,Halogen,1826
5,9,F,Fluorine,18.998403,90E050,[He]2s2 2p5,3.98,135.0,17.423,3.339,-1,Gas,53.53,85.03,0.001696,Halogen,1670
6,7,N,Nitrogen,14.007000,3050F8,[He] 2s2 2p3,3.04,155.0,14.534,NaN,"+5, +4, +3, +2, +1, -1, -2, -3",Gas,63.15,77.36,0.001251,Nonmetal,1772
7,17,Cl,Chlorine,35.450000,1FF01F,[Ne]3s2 3p5,3.16,175.0,12.968,3.617,"+7, +5, +1, -1",Gas,171.65,239.11,0.003214,Halogen,1774
8,6,C,Carbon,12.011000,909090,[He]2s2 2p2,2.55,170.0,11.260,1.263,"+4, +2, -4",Solid,3823.00,4098.00,2.267000,Nonmetal,Ancient
9,8,O,Oxygen,15.999000,FF0D0D,[He]2s2 2p4,3.44,152.0,13.618,1.461,-2,Gas,54.36,90.20,0.001429,Nonmetal,1774


In [11]:
at_number = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['AtomicNumber']
    
    at_number[key] = val

In [12]:
at_number

{'S': 16,
 'P': 15,
 'H': 1,
 'I': 53,
 'Br': 35,
 'F': 9,
 'N': 7,
 'Cl': 17,
 'C': 6,
 'O': 8,
 'B': 5,
 'Si': 14}

In [13]:
at_weight = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['AtomicMass']
    
    at_weight[key] = val

In [14]:
electro_neg = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['Electronegativity']
    
    electro_neg[key] = val

In [15]:
ion_energy = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['IonizationEnergy']
    
    ion_energy[key] = val

In [16]:
electron_aff = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['ElectronAffinity']
    
    electron_aff[key] = val

In [17]:
melting_p = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['MeltingPoint']
    
    melting_p[key] = val

In [18]:
boiling_p = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['BoilingPoint']
    
    boiling_p[key] = val

In [19]:
density = {}

for i in range(12):
    key = tmp.iloc[i]['Symbol']
    val = tmp.iloc[i]['Density']
    
    density[key] = val

In [20]:
# add atomic information to df_a as variables
df_a['at_n'] = df_a['3'].map(at_number)
df_a['at_w'] = df_a['3'].map(at_weight)
df_a['el_neg'] = df_a['3'].map(electro_neg)
df_a['ion_en'] = df_a['3'].map(ion_energy)
df_a['el_aff'] = df_a['3'].map(electron_aff)
df_a['mp'] = df_a['3'].map(melting_p)
df_a['bp'] = df_a['3'].map(boiling_p)
df_a['den'] = df_a['3'].map(density)
df_a.head()

,0,1,2,3,4,5,6,7,8,9,...,15,dist,at_n,at_w,el_neg,ion_en,el_aff,mp,bp,den
0,2.8972,-2.3256,0.4309,C,0,0,0,0,0,0,...,0,3.740034,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
1,3.3467,-1.0151,1.0796,C,0,0,0,0,0,0,...,0,3.660105,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
2,3.3365,0.2041,0.1293,C,0,0,1,0,0,0,...,0,3.345237,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
3,1.9424,0.6155,-0.3895,C,0,0,0,0,0,0,...,0,2.074480,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
4,1.0870,1.3235,0.6658,C,0,0,0,0,0,0,...,0,1.837528,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267


### (3) Bond Information
- bond length
- bond energy

In [21]:
path = 'data/mol_files/train_set/'

df_b = pd.read_csv(path + 'train_0_g_b.csv')
df_b.head()

,0,1,2,3
0,1,2,1,0
1,1,29,1,0
2,1,30,1,0
3,1,31,1,0
4,2,3,1,0


In [22]:
df_a.index = range(1, len(df_a)+1)

In [23]:
df_a.head()

,0,1,2,3,4,5,6,7,8,9,...,15,dist,at_n,at_w,el_neg,ion_en,el_aff,mp,bp,den
1,2.8972,-2.3256,0.4309,C,0,0,0,0,0,0,...,0,3.740034,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
2,3.3467,-1.0151,1.0796,C,0,0,0,0,0,0,...,0,3.660105,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
3,3.3365,0.2041,0.1293,C,0,0,1,0,0,0,...,0,3.345237,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
4,1.9424,0.6155,-0.3895,C,0,0,0,0,0,0,...,0,2.074480,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
5,1.0870,1.3235,0.6658,C,0,0,0,0,0,0,...,0,1.837528,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267


#### bond length

In [24]:
at_list_1 = []
at_list_2 = []
len_list = []

for i in df_b.index:
    
    idx_1 = df_b.iloc[i, 0]
    idx_2 = df_b.iloc[i, 1]
    
    atom_1 = df_a.loc[idx_1]
    atom_2 = df_a.loc[idx_2]
    
    sym_1 = atom_1['3']
    sym_2 = atom_2['3']
    bond_len = np.sqrt((atom_1['0'] - atom_2['0'])**2 + (atom_1['1'] - atom_2['1'])**2 + (atom_1['2'] - atom_2['2'])**2)
    
    at_list_1.append(sym_1)
    at_list_2.append(sym_2)
    len_list.append(bond_len)

In [25]:
bond_df = pd.DataFrame()
bond_df['at_1'] = at_list_1
bond_df['at_2'] = at_list_2
bond_df['type'] = df_b['2']
bond_df['len'] = len_list

In [26]:
bond_df.head()

# bond_df contains bond information such as bond length and bond energy
# interpreted from bonds block data in df_b
# thus, len(bond_df) == len(df_b)

,at_1,at_2,type,len
0,C,C,1,1.529795
1,C,H,1,1.101256
2,C,H,1,1.103473
3,C,H,1,1.101764
4,C,C,1,1.545840


#### bond energy

In [27]:
table = pd.read_html('https://www.wiredchemist.com/chemistry/data/bond_energies_lengths.html')

In [28]:
t1 = table[0]
t2 = table[1]
t3 = table[2]
t4 = table[3]
t5 = table[4].loc[0:22]
t6 = table[5].drop(4)
t7 = table[6]

In [29]:
t12 = pd.concat([t1, t2])
t34 = pd.concat([t3, t4])
t56 = pd.concat([t5, t6])

In [30]:
t1234 = pd.concat([t12, t34])
t567 = pd.concat([t56, t7])
t_all = pd.concat([t1234, t567])

In [31]:
en_df = t_all.iloc[:, 0:2].dropna().reset_index(drop=True)
en_df.columns = ['bond', 'energy']
en_df

,bond,energy
0,H-H,432
1,H-B,389
2,H-C,411
3,H-Si,318
4,H-Ge,288
...,...,...
101,At-At,116
102,I-O,201
103,I-F,273
104,I-Cl,208


In [32]:
en_df['type'] = 1
en_df['at_1'] = 1
en_df['at_2'] = 1
en_df.head()

,bond,energy,type,at_1,at_2
0,H-H,432,1,1,1
1,H-B,389,1,1,1
2,H-C,411,1,1,1
3,H-Si,318,1,1,1
4,H-Ge,288,1,1,1


In [33]:
for idx, bond in enumerate(en_df['bond']):
    if '-' in bond:
        at_1 = bond.split('-')[0]
        at_2 = bond.split('-')[1]
        en_df.loc[idx, 'at_1'] = at_1
        en_df.loc[idx, 'at_2'] = at_2
    elif '=' in bond:
        at_1 = bond.split('=')[0]
        at_2 = bond.split('=')[1]
        en_df.loc[idx, 'type'] = 2
        en_df.loc[idx, 'at_1'] = at_1
        en_df.loc[idx, 'at_2'] = at_2
    elif '≡' in bond:
        at_1 = bond.split('≡')[0]
        at_2 = bond.split('≡')[1]
        en_df.loc[idx, 'type'] = 3
        en_df.loc[idx, 'at_1'] = at_1
        en_df.loc[idx, 'at_2'] = at_2

In [34]:
en_df.head()

,bond,energy,type,at_1,at_2
0,H-H,432,1,H,H
1,H-B,389,1,H,B
2,H-C,411,1,H,C
3,H-Si,318,1,H,Si
4,H-Ge,288,1,H,Ge


In [35]:
bond_df['bond_en'] = 1
bond_df.head()

,at_1,at_2,type,len,bond_en
0,C,C,1,1.529795,1
1,C,H,1,1.101256,1
2,C,H,1,1.103473,1
3,C,H,1,1.101764,1
4,C,C,1,1.545840,1


In [36]:
set_list = []

for i in bond_df.index:
    at_set = {bond_df.loc[i, 'at_1'], bond_df.loc[i, 'at_2']}
    set_list.append(at_set)

In [37]:
set_list[:10]

[{'C'},
 {'C', 'H'},
 {'C', 'H'},
 {'C', 'H'},
 {'C'},
 {'C', 'H'},
 {'C', 'H'},
 {'C'},
 {'C', 'N'},
 {'C', 'H'}]

In [38]:
for idx, bond in enumerate(set_list):
    
    for j in en_df.index:
        at_set = {en_df.loc[j, 'at_1'], en_df.loc[j, 'at_2']}
        
        if (bond == at_set) and (bond_df.loc[idx, 'type'] == en_df.loc[j, 'type']):
            bond_df.loc[idx, 'bond_en'] = en_df.loc[j, 'energy']

In [39]:
bond_df.head()

,at_1,at_2,type,len,bond_en
0,C,C,1,1.529795,346
1,C,H,1,1.101256,411
2,C,H,1,1.103473,411
3,C,H,1,1.101764,411
4,C,C,1,1.545840,346


## Aggregation of Created Variables
- np.mean

In [40]:
df_a.head()

,0,1,2,3,4,5,6,7,8,9,...,15,dist,at_n,at_w,el_neg,ion_en,el_aff,mp,bp,den
1,2.8972,-2.3256,0.4309,C,0,0,0,0,0,0,...,0,3.740034,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
2,3.3467,-1.0151,1.0796,C,0,0,0,0,0,0,...,0,3.660105,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
3,3.3365,0.2041,0.1293,C,0,0,1,0,0,0,...,0,3.345237,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
4,1.9424,0.6155,-0.3895,C,0,0,0,0,0,0,...,0,2.074480,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267
5,1.0870,1.3235,0.6658,C,0,0,0,0,0,0,...,0,1.837528,6,12.011,2.55,11.26,1.263,3823.0,4098.0,2.267


In [41]:
bond_df.head()

,at_1,at_2,type,len,bond_en
0,C,C,1,1.529795,346
1,C,H,1,1.101256,411
2,C,H,1,1.103473,411
3,C,H,1,1.101764,411
4,C,C,1,1.545840,346


In [42]:
df_a.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'dist', 'at_n', 'at_w', 'el_neg', 'ion_en', 'el_aff',
       'mp', 'bp', 'den'],
      dtype='object')

In [43]:
bond_df.columns

Index(['at_1', 'at_2', 'type', 'len', 'bond_en'], dtype='object')

In [44]:
a_cols = ['at_n', 'at_w', 'el_neg', 'ion_en', 'el_aff', 'mp', 'bp', 'den']
b_cols = ['len', 'bond_en']

In [45]:
# use np.mean as agg function since none of the atoms in a molecule are considered an outlier

data = []
avg_dic = {}

for i in a_cols:
    avg = np.mean(df_a[i])
    avg_dic[i] = avg

for i in b_cols:
    avg = np.mean(bond_df[i])
    avg_dic[i] = avg

data.append(avg_dic)

In [46]:
pd.DataFrame(data)

,at_n,at_w,el_neg,ion_en,el_aff,mp,bp,den,len,bond_en
0,3.849057,7.234868,2.479811,12.726113,1.02434,1528.964906,1644.488302,0.898466,1.274554,416.872727
